In [1]:
import os
os.getcwd()
os.chdir("H:\pyspark_advanced-coding_interview")
os.getcwd()

'H:\\pyspark_advanced-coding_interview'

In [2]:
from pyspark.sql import SparkSession

# Create a Spark session with optimized settings
spark = (
    SparkSession.builder 
    .appName("OptimizedLocalSpark") 
    .config("spark.driver.memory", "8g")        
    .config("spark.executor.memory", "8g")    
    .config("spark.executor.cores", "4")       
    .config("spark.cores.max", "12")           
    .config("spark.sql.shuffle.partitions", "28")  
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") 
    .getOrCreate()
)
sc = spark.sparkContext

In [1]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("AdvancedRegexRealTimeExamples").getOrCreate()

# Sample Data: Various strings for complex regex processing
data = [
    (1, "Loving the new #iPhone13! #Apple #Tech", "+1-800-555-1234", "USER001", "Hello [world]!", "Let's eat, Grandma!", "This is the end.", "IP found: 192.168.1.1", "00:1A:2B:3C:4D:5E", "My credit card is 1234 5678 9012 3456", "alice@example.com", "550e8400-e29b-41d4-a716-446655440000", "/path/to/file/document.txt", "Café au lait", "https://www.example.com?user=alice&lang=en", "<p>Hello <b>world</b>!</p>", "userEmail", "+1-123-456-7890", "4111 1111 1111 1111", "apple orange banana", "#FF5733"),
    (2, "Just released #PS5! It's amazing!", "(800)555-9876", "ADMIN100", "Extract [this part]", "Why so serious?!", "Let's move on.", "Connect to 10.0.0.1", "01:23:45:67:89:ab", "Use pin: 0987", "bob.smith@yahoo.com", "6ba7b810-9dad-11d1-80b4-00c04fd430c8", "/files/uploads/image.png", "naïve approach", "https://example.com?page=1&limit=10", "<div>Sample <i>text</i></div>", "lowerCamelCase", "+44 20 7946 0958", "4000 1234 5678 9010", "umbrella island", "#aB12CD"),
    (3, "#Travel #Adventure is life!", "800.555.4321", "DEVELOPER", "Between [brackets]", "Wait! What?!!", "No suffix here", "Found: 172.16.254.1", "AB-CD-EF-12-34-56", "My pass: 5678", "contact@domain.com", "9f8d0db1-d8b0-4b53-991e-99300dbdb5c1", "/user/photos/pic.jpg", "façade", "https://shop.com/products?id=1001", "<h1>Welcome</h1>", "camelCaseExample", "555-222-3333", "5105 1051 0510 0510", "orange apple", "#BADA55"),
    # More records can be added similarly
]

# Create DataFrame
df = spark.createDataFrame(data, ["id", "post", "phone", "prefix", "between_chars", "punctuated", "suffix_check", "text_with_ip", "mac_address", "text_with_digits", "email", "uuid", "file_path", "non_ascii", "url_with_params", "html_content", "camel_case", "phone_area", "credit_card", "vowel_words", "hex_color"])

# Create a Temporary View for Spark SQL
df.createOrReplaceTempView("regex_advanced_examples")
df.cache()
# Show the Original DataFrame
df.show(truncate=False)


+---+--------------------------------------+---------------+---------+-------------------+-------------------+----------------+---------------------+-----------------+-------------------------------------+-------------------+------------------------------------+--------------------------+--------------+------------------------------------------+-----------------------------+----------------+----------------+-------------------+-------------------+---------+
|id |post                                  |phone          |prefix   |between_chars      |punctuated         |suffix_check    |text_with_ip         |mac_address      |text_with_digits                     |email              |uuid                                |file_path                 |non_ascii     |url_with_params                           |html_content                 |camel_case      |phone_area      |credit_card        |vowel_words        |hex_color|
+---+--------------------------------------+---------------+---------+------

# Extract Hashtags from a Social Media Post

In [2]:
res = spark.sql (""" 
  
SELECT id, post, regexp_extract_all(post, '#[A-Za-z0-9]+', 0) AS hashtags FROM regex_advanced_examples;
              
                   
                   """)

res.show()


from pyspark.sql.functions import regexp_extract

df.withColumn("hashtags", regexp_extract("post", "#[A-Za-z0-9]+", 0)).show(truncate=False)


+---+--------------------+--------------------+
| id|                post|            hashtags|
+---+--------------------+--------------------+
|  1|Loving the new #i...|[#iPhone13, #Appl...|
|  2|Just released #PS...|              [#PS5]|
|  3|#Travel #Adventur...|[#Travel, #Advent...|
+---+--------------------+--------------------+



# Validate Phone Numbers in Various Formats

In [4]:
df.withColumn("phone_status", df["phone"].rlike("^(\\+?\\d{1,3}[- .]?\\(\\d{3}\\)[- .]?|\\d{3}[- .])\\d{3}[- .]?\\d{4}$")).show(truncate=False)


+---+--------------------------------------+---------------+---------+-------------------+-------------------+----------------+---------------------+-----------------+-------------------------------------+-------------------+------------------------------------+--------------------------+--------------+------------------------------------------+-----------------------------+----------------+----------------+-------------------+-------------------+---------+------------+
|id |post                                  |phone          |prefix   |between_chars      |punctuated         |suffix_check    |text_with_ip         |mac_address      |text_with_digits                     |email              |uuid                                |file_path                 |non_ascii     |url_with_params                           |html_content                 |camel_case      |phone_area      |credit_card        |vowel_words        |hex_color|phone_status|
+---+--------------------------------------+------

In [3]:
res1 = spark.sql (""" 
  
SELECT id, phone, CASE WHEN phone RLIKE '^(\\+?\\d{1,3}[- .]?\\(\\d{3}\\)[- .]?|\\d{3}[- .])\\d{3}[- .]?\\d{4}$' THEN 'Valid' ELSE 'Invalid' END AS phone_status FROM regex_advanced_examples;
  
                   """)

res1.show()

Py4JJavaError: An error occurred while calling o47.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 40) (localhost executor driver): java.util.regex.PatternSyntaxException: Dangling meta character '+' near index 2
^(+?d{1,3}[- .]?(d{3})[- .]?|d{3}[- .])d{3}[- .]?d{4}$
  ^
	at java.base/java.util.regex.Pattern.error(Pattern.java:2015)
	at java.base/java.util.regex.Pattern.sequence(Pattern.java:2190)
	at java.base/java.util.regex.Pattern.expr(Pattern.java:2056)
	at java.base/java.util.regex.Pattern.group0(Pattern.java:3045)
	at java.base/java.util.regex.Pattern.sequence(Pattern.java:2111)
	at java.base/java.util.regex.Pattern.expr(Pattern.java:2056)
	at java.base/java.util.regex.Pattern.compile(Pattern.java:1778)
	at java.base/java.util.regex.Pattern.<init>(Pattern.java:1427)
	at java.base/java.util.regex.Pattern.compile(Pattern.java:1068)
	at org.apache.sp

# Check if a String Starts with a Specific Prefix

In [6]:
res2 = spark.sql (""" 
  
SELECT id, prefix, CASE WHEN prefix RLIKE '^USER' THEN 'Starts with USER' ELSE 'Different prefix' END AS prefix_check FROM regex_advanced_examples;

  
                   """)

res2.show()


df.withColumn("prefix_check", df["prefix"].rlike("^USER")).show(truncate=False)


+---+---------+----------------+
| id|   prefix|    prefix_check|
+---+---------+----------------+
|  1|  USER001|Starts with USER|
|  2| ADMIN100|Different prefix|
|  3|DEVELOPER|Different prefix|
+---+---------+----------------+

+---+--------------------------------------+---------------+---------+-------------------+-------------------+----------------+---------------------+-----------------+-------------------------------------+-------------------+------------------------------------+--------------------------+--------------+------------------------------------------+-----------------------------+----------------+----------------+-------------------+-------------------+---------+------------+
|id |post                                  |phone          |prefix   |between_chars      |punctuated         |suffix_check    |text_with_ip         |mac_address      |text_with_digits                     |email              |uuid                                |file_path                 |non_

# Extract Everything Between Two Characters

In [9]:

from pyspark.sql.functions import regexp_extract


res3 = spark.sql (""" 
  
SELECT id, between_chars, regexp_extract(between_chars, '\\[(.*?)\\]', 1) AS extracted_part FROM regex_advanced_examples;


  
                   """)

res3.show()

df.withColumn("extracted_part", regexp_extract("between_chars", "\\[(.*?)\\]", 1)).show(truncate=False)


+---+-------------------+--------------+
| id|      between_chars|extracted_part|
+---+-------------------+--------------+
|  1|     Hello [world]!|              |
|  2|Extract [this part]|              |
|  3| Between [brackets]|              |
+---+-------------------+--------------+

+---+--------------------------------------+---------------+---------+-------------------+-------------------+----------------+---------------------+-----------------+-------------------------------------+-------------------+------------------------------------+--------------------------+--------------+------------------------------------------+-----------------------------+----------------+----------------+-------------------+-------------------+---------+--------------+
|id |post                                  |phone          |prefix   |between_chars      |punctuated         |suffix_check    |text_with_ip         |mac_address      |text_with_digits                     |email              |uuid      

# Remove Punctuation from a String

In [13]:

from pyspark.sql.functions import regexp_extract, regexp_replace


res5 = spark.sql (""" 
  
SELECT id, punctuated, regexp_replace(punctuated, '[\\p{Punct}]', '') AS no_punctuation FROM regex_advanced_examples;


  
                   """)

res5.show()


from pyspark.sql.functions import regexp_replace

df.select("punctuated").withColumn("no_punctuation", regexp_replace("punctuated", "[\\p{Punct}]", "")).show(truncate=False)


+---+-------------------+----------------+
| id|         punctuated|  no_punctuation|
+---+-------------------+----------------+
|  1|Let's eat, Grandma!|Le's ea, Gradma!|
|  2|   Why so serious?!| Why so serios?!|
|  3|      Wait! What?!!|     Wai! Wha?!!|
+---+-------------------+----------------+

+-------------------+----------------+
|punctuated         |no_punctuation  |
+-------------------+----------------+
|Let's eat, Grandma!|Lets eat Grandma|
|Why so serious?!   |Why so serious  |
|Wait! What?!!      |Wait What       |
+-------------------+----------------+



# Check if a String Ends with a Specific Suffix

In [15]:
res6 = spark.sql (""" 
  
SELECT id, suffix_check, 
CASE WHEN suffix_check RLIKE 'end\\.$' THEN 'Ends with end.' ELSE 'Different suffix' END AS suffix_validation 
FROM regex_advanced_examples;



  
                   """)

res6.show()


df.select("suffix_check").withColumn("suffix_validation", df["suffix_check"].rlike("end\\.$")).show(truncate=False)


+---+----------------+-----------------+
| id|    suffix_check|suffix_validation|
+---+----------------+-----------------+
|  1|This is the end.|   Ends with end.|
|  2|  Let's move on.| Different suffix|
|  3|  No suffix here| Different suffix|
+---+----------------+-----------------+

+----------------+-----------------+
|suffix_check    |suffix_validation|
+----------------+-----------------+
|This is the end.|true             |
|Let's move on.  |false            |
|No suffix here  |false            |
+----------------+-----------------+



# Extract IPs from a Text Blob

In [18]:
res7 = spark.sql (""" 
  
SELECT id, text_with_ip, regexp_extract(text_with_ip, '\\b\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\b', 0) AS extracted_ip 

FROM regex_advanced_examples;



  
                   """)

res7.show()


df.select("text_with_ip").withColumn("extracted_ip", regexp_extract("text_with_ip", "\\b\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\b", 0)).show(truncate=False)


+---+--------------------+------------+
| id|        text_with_ip|extracted_ip|
+---+--------------------+------------+
|  1|IP found: 192.168...|            |
|  2| Connect to 10.0.0.1|            |
|  3| Found: 172.16.254.1|            |
+---+--------------------+------------+

+---------------------+------------+
|text_with_ip         |extracted_ip|
+---------------------+------------+
|IP found: 192.168.1.1|192.168.1.1 |
|Connect to 10.0.0.1  |10.0.0.1    |
|Found: 172.16.254.1  |172.16.254.1|
+---------------------+------------+



# Validate MAC Addresses

In [20]:
res8 = spark.sql (""" 
  
SELECT id, mac_address, CASE WHEN mac_address RLIKE '^([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})$' THEN 'Valid' ELSE 'Invalid' END AS mac_status FROM regex_advanced_examples;




  
                   """)

res8.show()


df.select("mac_address").withColumn("mac_status", df["mac_address"].rlike("^([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})$")).show(truncate=False)


+---+-----------------+----------+
| id|      mac_address|mac_status|
+---+-----------------+----------+
|  1|00:1A:2B:3C:4D:5E|     Valid|
|  2|01:23:45:67:89:ab|     Valid|
|  3|AB-CD-EF-12-34-56|     Valid|
+---+-----------------+----------+

+-----------------+----------+
|mac_address      |mac_status|
+-----------------+----------+
|00:1A:2B:3C:4D:5E|true      |
|01:23:45:67:89:ab|true      |
|AB-CD-EF-12-34-56|true      |
+-----------------+----------+



#  Replace All Digits with Asterisks

In [23]:
res9 = spark.sql (""" 
  
SELECT id, text_with_digits, regexp_replace(text_with_digits, '\\d', '*') AS obfuscated_text FROM regex_advanced_examples;

                   """)

res9.show()


df.select("text_with_digits").withColumn("obfuscated_text", regexp_replace("text_with_digits", "\\d", "*")).show(truncate=False)


+---+--------------------+--------------------+
| id|    text_with_digits|     obfuscated_text|
+---+--------------------+--------------------+
|  1|My credit card is...|My cre*it car* is...|
|  2|       Use pin: 0987|       Use pin: 0987|
|  3|       My pass: 5678|       My pass: 5678|
+---+--------------------+--------------------+

+-------------------------------------+-------------------------------------+
|text_with_digits                     |obfuscated_text                      |
+-------------------------------------+-------------------------------------+
|My credit card is 1234 5678 9012 3456|My credit card is **** **** **** ****|
|Use pin: 0987                        |Use pin: ****                        |
|My pass: 5678                        |My pass: ****                        |
+-------------------------------------+-------------------------------------+



# Extract the Username from an Email

In [24]:
spark.sql("select * from regex_advanced_examples").show()

+---+--------------------+---------------+---------+-------------------+-------------------+----------------+--------------------+-----------------+--------------------+-------------------+--------------------+--------------------+--------------+--------------------+--------------------+----------------+----------------+-------------------+-------------------+---------+
| id|                post|          phone|   prefix|      between_chars|         punctuated|    suffix_check|        text_with_ip|      mac_address|    text_with_digits|              email|                uuid|           file_path|     non_ascii|     url_with_params|        html_content|      camel_case|      phone_area|        credit_card|        vowel_words|hex_color|
+---+--------------------+---------------+---------+-------------------+-------------------+----------------+--------------------+-----------------+--------------------+-------------------+--------------------+--------------------+--------------+--------

In [26]:
res10 = spark.sql (""" 
  
SELECT id, email, regexp_extract(email, '^[A-Za-z0-9._%+-]+', 0) AS username FROM regex_advanced_examples;

                   """)

res10.show()

df.select("email").withColumn("username", regexp_extract("email", "^[A-Za-z0-9._%+-]+", 0)).show(truncate=False)

+---+-------------------+---------+
| id|              email| username|
+---+-------------------+---------+
|  1|  alice@example.com|    alice|
|  2|bob.smith@yahoo.com|bob.smith|
|  3| contact@domain.com|  contact|
+---+-------------------+---------+

+-------------------+---------+
|email              |username |
+-------------------+---------+
|alice@example.com  |alice    |
|bob.smith@yahoo.com|bob.smith|
|contact@domain.com |contact  |
+-------------------+---------+



# Check if String Matches a UUID Pattern

In [28]:
res11 = spark.sql (""" 
  
SELECT id, uuid, CASE WHEN uuid RLIKE '^[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{12}$' THEN 'Valid' ELSE 'Invalid' END AS uuid_check FROM regex_advanced_examples;


                   """)

res11.show()

df.select("uuid").withColumn("uuid_check", df["uuid"].rlike("^[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{12}$")).show(truncate=False)


+---+--------------------+----------+
| id|                uuid|uuid_check|
+---+--------------------+----------+
|  1|550e8400-e29b-41d...|     Valid|
|  2|6ba7b810-9dad-11d...|     Valid|
|  3|9f8d0db1-d8b0-4b5...|     Valid|
+---+--------------------+----------+

+------------------------------------+----------+
|uuid                                |uuid_check|
+------------------------------------+----------+
|550e8400-e29b-41d4-a716-446655440000|true      |
|6ba7b810-9dad-11d1-80b4-00c04fd430c8|true      |
|9f8d0db1-d8b0-4b53-991e-99300dbdb5c1|true      |
+------------------------------------+----------+



# Extract the Filename from a File Path

In [31]:
res12 = spark.sql (""" 
  
SELECT id, file_path, regexp_extract(file_path, '[^/]+$', 0) AS filename FROM regex_advanced_examples;

                   """)

res12.show()

df.select("file_path").withColumn("filename", regexp_extract("file_path", "[^/]+$", 0)).show(truncate=False)


+---+--------------------+------------+
| id|           file_path|    filename|
+---+--------------------+------------+
|  1|/path/to/file/doc...|document.txt|
|  2|/files/uploads/im...|   image.png|
|  3|/user/photos/pic.jpg|     pic.jpg|
+---+--------------------+------------+

+--------------------------+------------+
|file_path                 |filename    |
+--------------------------+------------+
|/path/to/file/document.txt|document.txt|
|/files/uploads/image.png  |image.png   |
|/user/photos/pic.jpg      |pic.jpg     |
+--------------------------+------------+



# Replace Non-ASCII Characters

In [33]:
res13 = spark.sql (""" 
  
SELECT id, non_ascii, regexp_replace(non_ascii, '[^\\x00-\\x7F]', '') AS ascii_only FROM regex_advanced_examples;

                   """)

res13.show()

df.select("non_ascii").withColumn("ascii_only", regexp_replace("non_ascii", "[^\\x00-\\x7F]", "")).show(truncate=False)


+---+--------------+------------+
| id|     non_ascii|  ascii_only|
+---+--------------+------------+
|  1|  Café au lait|   Cafaulait|
|  2|naïve approach|naveapproach|
|  3|        façade|       faade|
+---+--------------+------------+

+--------------+-------------+
|non_ascii     |ascii_only   |
+--------------+-------------+
|Café au lait  |Caf au lait  |
|naïve approach|nave approach|
|façade        |faade        |
+--------------+-------------+



# Extract URL Parameters from a Query String

In [35]:
spark.sql("select * from regex_advanced_examples").show()

+---+--------------------+---------------+---------+-------------------+-------------------+----------------+--------------------+-----------------+--------------------+-------------------+--------------------+--------------------+--------------+--------------------+--------------------+----------------+----------------+-------------------+-------------------+---------+
| id|                post|          phone|   prefix|      between_chars|         punctuated|    suffix_check|        text_with_ip|      mac_address|    text_with_digits|              email|                uuid|           file_path|     non_ascii|     url_with_params|        html_content|      camel_case|      phone_area|        credit_card|        vowel_words|hex_color|
+---+--------------------+---------------+---------+-------------------+-------------------+----------------+--------------------+-----------------+--------------------+-------------------+--------------------+--------------------+--------------+--------

In [38]:
# res14 = spark.sql (""" 
  
# SELECT id, url_with_params, regexp_extract(url_with_params, '\\?.*', 0) AS query_params FROM regex_advanced_examples;


#                    """)

# res14.show()


df.select("url_with_params").withColumn("query_params", regexp_extract("url_with_params", "\\?.*", 0)).show(truncate=False)


+------------------------------------------+-------------------+
|url_with_params                           |query_params       |
+------------------------------------------+-------------------+
|https://www.example.com?user=alice&lang=en|?user=alice&lang=en|
|https://example.com?page=1&limit=10       |?page=1&limit=10   |
|https://shop.com/products?id=1001         |?id=1001           |
+------------------------------------------+-------------------+



# Remove All HTML Tags

In [39]:
res15 = spark.sql (""" 
  
SELECT id, html_content, regexp_replace(html_content, '<[^>]+>', '') AS text_content FROM regex_advanced_examples;


                   """)

res15.show()

df.select("html_content").withColumn("text_content", regexp_replace("html_content", "<[^>]+>", "")).show(truncate=False)


+---+--------------------+------------+
| id|        html_content|text_content|
+---+--------------------+------------+
|  1|<p>Hello <b>world...|Hello world!|
|  2|<div>Sample <i>te...| Sample text|
|  3|    <h1>Welcome</h1>|     Welcome|
+---+--------------------+------------+

+-----------------------------+------------+
|html_content                 |text_content|
+-----------------------------+------------+
|<p>Hello <b>world</b>!</p>   |Hello world!|
|<div>Sample <i>text</i></div>|Sample text |
|<h1>Welcome</h1>             |Welcome     |
+-----------------------------+------------+



# Convert camelCase to snake_case

In [41]:
df.select("camel_case").withColumn("snake_case", regexp_replace("camel_case", "(?<=[a-z])([A-Z])", "_$1")).show(truncate=False)




+----------------+------------------+
|camel_case      |snake_case        |
+----------------+------------------+
|userEmail       |user_Email        |
|lowerCamelCase  |lower_Camel_Case  |
|camelCaseExample|camel_Case_Example|
+----------------+------------------+



# Extract Area Code from a Phone Number

In [44]:
# res16 = spark.sql (""" 
# SELECT id, phone_area, regexp_extract(phone_area, '\\(?\\d{3}\\)?', 0) AS area_code FROM regex_advanced_examples;

#                    """)

# res16.show()

df.select("phone_area").withColumn("area_code", regexp_extract("phone_area", "\\(?\\d{3}\\)?", 0)).show(truncate=False)


+----------------+---------+
|phone_area      |area_code|
+----------------+---------+
|+1-123-456-7890 |123      |
|+44 20 7946 0958|794      |
|555-222-3333    |555      |
+----------------+---------+



# Validate Credit Card Numbers

In [45]:
res17 = spark.sql (""" 
SELECT id, credit_card, CASE WHEN credit_card RLIKE '^\\d{4}[- ]?\\d{4}[- ]?\\d{4}[- ]?\\d{4}$' THEN 'Valid' ELSE 'Invalid' END AS card_status FROM regex_advanced_examples;


                   """)

res17.show()

df.select("credit_card").withColumn("card_status", df["credit_card"].rlike("^\\d{4}[- ]?\\d{4}[- ]?\\d{4}[- ]?\\d{4}$")).show(truncate=False)



+---+-------------------+-----------+
| id|        credit_card|card_status|
+---+-------------------+-----------+
|  1|4111 1111 1111 1111|    Invalid|
|  2|4000 1234 5678 9010|    Invalid|
|  3|5105 1051 0510 0510|    Invalid|
+---+-------------------+-----------+

+-------------------+-----------+
|credit_card        |card_status|
+-------------------+-----------+
|4111 1111 1111 1111|true       |
|4000 1234 5678 9010|true       |
|5105 1051 0510 0510|true       |
+-------------------+-----------+



# Extract All Words Starting with a Vowel

In [47]:
res18 = spark.sql (""" 
SELECT id, vowel_words, regexp_extract_all(vowel_words, '\\b[AEIOUaeiou][a-zA-Z]*\\b', 0) AS vowel_start_words FROM regex_advanced_examples;


                   """)

res18.show()

df.select("vowel_words").withColumn("vowel_start_words", regexp_extract("vowel_words", "\\b[AEIOUaeiou][a-zA-Z]*\\b", 0)).show(truncate=False)


+---+-------------------+-----------------+
| id|        vowel_words|vowel_start_words|
+---+-------------------+-----------------+
|  1|apple orange banana|               []|
|  2|    umbrella island|               []|
|  3|       orange apple|               []|
+---+-------------------+-----------------+

+-------------------+-----------------+
|vowel_words        |vowel_start_words|
+-------------------+-----------------+
|apple orange banana|apple            |
|umbrella island    |umbrella         |
|orange apple       |orange           |
+-------------------+-----------------+

